# Objet : Analyse des relations entre champs des données IRVE

## Contexte
- clarification du rôle des modèles de données dans les jeux de données (cf mise à jour récente des [guides data.gouv](https://guides.etalab.gouv.fr/qualite/documenter-les-donnees/))
- intégration d'une propriété "relationship" dans les schémas de données ([issue TableSchema](https://github.com/frictionlessdata/specs/issues/803) en cours de validation)
- création d'outils de contrôle des relations entre champs des jeux de données tabulaires (cf usage ci-dessous)

## Objectifs
- valider sur un cas réel l'utilisation d'un modèle de données en complément d'un schéma de données
- identifier les apports que pourraient avoir les contrôles de validation des relations entre champs

## Résultats
- la formalisation d'un modèle de données facilite la compréhension des données et des relations entre champs
- l'outil de contrôle permet d'améliorer significativement la qualité des données par la détection d'incohérences de relations
- l'analyse a posteriori permet de (re)trouver la logique de dépendance entre les colonnes qui minimise les incohérences
- les incohérences détectées sur le jeu de données IRVE restent faibles (inférieures à 2% des point de charge documentés - voir chapitre 4)

## Suite à donner
- Mettre à jour, valider et publier le modèle de données IRVE
- Définir les contrôles supplémentaires à intégrer pour toutes nouvelles données ainsi que pour le jeu complet
- Mettre en oeuvre les outils de contrôle

## Evolutions possibles 
- Ajouter dans les guides d'Etalab un guide pour les modèles de données 
- Intégrer dans les schémas de données la propriété "relationship" en cours de validation,
- Définir un indicateur qui mesure l'écart (existant / attendu) des relations entre champs

## Sommaire
*(liens actifs sur jupyter Notebook ou Nbviewer)*
- [1 - modèle de données](#1---modèle-de-données)
- [2 - Initialisation](#2---Initialisation)
- [3 - Séparation des pdc itinerance et hors itinerance](#3---Séparation-des-pdc-itinerance-et-hors-itinerance)
- [4 - Bilan initial intégrité](#4---Bilan-initial-intégrité)
- [5 - Séparation doublons pdc - date de maj](#5---Séparation-doublons-pdc---date-de-maj)
- [6 - Cohérence date maj stations](#6---Cohérence-date-maj-stations)
- [7 - Synthèse](#7---Synthèse)
- [8 - Exemples d erreurs résiduelles](#8---Exemples-d-erreurs-résiduelles)
- [Annexe : Initialisation du modèle de données 'mermaid'](#Annexe-:-Initialisation-du-modèle-de-données-'mermaid')

Ce Notebook peut être consulté sur [nbviewer](http://nbviewer.org/github/loco-philippe/Environmental-Sensing/tree/main/python/Validation/irve)

données utilisées : https://www.data.gouv.fr/fr/datasets/fichier-consolide-des-bornes-de-recharge-pour-vehicules-electriques/    
fichier : "*consolidation-etalab-schema-irve-statique-v-2.2.0-20230303.csv*"

-----------------------
# 1 - modèle de données

Le modèle de données proposé ci-dessous est construit sur la base du schéma de données mis à disposition et du contenu du jeu de données.    
Il est à consolider en fonction de l'expertise des concepteurs et réutilisateurs (voir [guide méthodologique](https://github.com/loco-philippe/Environmental-Sensing/blob/main/property_relationship/FR_methodology.ipynb)).

*Notation:*
- *M : Mandatory - documentation obligatoire*
- *PK : Primary Key - identifiant unique de l'entité*
- *Root : champ fictif associé à une ligne du tableau*

In [3]:
# la variable 'irve' est définie en fin de Notebook
mermaid(irve)

In [4]:
def analyse_integrite(data):
    analyse = Analysis(data)
    dic_res = analyse.check_relationship(fields)
    data['ok'] = True
    for name, lis in dic_res.items():
        data[name] = True
        data.loc[lis, name] = False
        data['ok'] = data['ok'] & data[name]
        print('{:<50} {:>5}'.format(name, len(data) - data[name].sum()))
    return dic_res

------
# 2 - Initialisation
## initialisation des données
- lecture du fichier issu de l'api

In [ ]:
from collections import Counter
from observation import Ilist, Iindex, Analysis
import pandas as pd
res = {}
chemin = 'https://raw.githubusercontent.com/loco-philippe/Environmental-Sensing/main/python/Validation/irve/'
res['date'] = '2023-03-03'
file = 'consolidation-etalab-schema-irve-statique-v-2.2.0-20230303.csv'
irve = pd.read_csv(chemin + file, sep=',', low_memory=False)
res['len_irve'] = len(irve)
print('nombre de lignes : ', res['len_irve']) 

## schéma de données
Le schéma de données restreint à la propriété 'relationship' et construit à partir du modèle de données est le suivants :

In [ ]:
fields = [
 
 # relation unicité des pdl
 { "name": "index",
   "relationship" : { "parent" : "id_pdc_itinerance", "link" : "coupled" }},   
 # relations inter entités
 { "name": "contact_operateur",
   "relationship" : { "parent" : "id_station_itinerance", "link" : "derived" }},
 { "name": "nom_enseigne",
   "relationship" : { "parent" : "id_station_itinerance", "link" : "derived" }},
 { "name": "coordonneesXY",
   "relationship" : { "parent" : "id_station_itinerance", "link" : "derived" }},
 { "name": "id_station_itinerance",
   "relationship" : { "parent" : "id_pdc_itinerance",     "link" : "derived" }},
 # relations intra entité - station
 { "name": "nom_station",
   "relationship" : { "parent" : "id_station_itinerance", "link" : "derived" }},
 { "name": "implantation_station",
   "relationship" : { "parent" : "id_station_itinerance", "link" : "derived" }},
 { "name": "nbre_pdc",
   "relationship" : { "parent" : "id_station_itinerance", "link" : "derived" }},
 { "name": "condition_acces",
   "relationship" : { "parent" : "id_station_itinerance", "link" : "derived" }},
 { "name": "horaires",
   "relationship" : { "parent" : "id_station_itinerance", "link" : "derived" }},
 { "name": "station_deux_roues",
   "relationship" : { "parent" : "id_station_itinerance", "link" : "derived" }},
 # relations intra entité - localisation
 { "name": "adresse_station",
   "relationship" : { "parent" : "coordonneesXY",         "link" : "derived" }} ]

In [ ]:
relations = ['index', 'contact_operateur', 'nom_enseigne', 'coordonneesXY', 'adresse_station', 'id_station_itinerance', 
             'nom_station', 'implantation_station', 'nbre_pdc', 'condition_acces', 'horaires', 'station_deux_roues', 
             'id_pdc_itinerance', 'date_maj', 'last_modified']
champs =  ['contact_operateur', 'nom_enseigne', 'coordonneesXY', 'adresse_station', 'id_station_itinerance', 'nom_station',
           'implantation_station', 'nbre_pdc', 'condition_acces', 'horaires', 'station_deux_roues', 'id_pdc_itinerance', 
           'puissance_nominale', 'prise_type_ef', 'prise_type_2', 'prise_type_combo_ccs', 'prise_type_chademo', 
           'prise_type_autre', 'paiement_acte', 'paiement_autre', 'reservation',  'accessibilite_pmr', 'restriction_gabarit', 
           'date_maj', 'last_modified']

-----------------------
## 3 - Séparation des pdc itinerance et hors itinerance
- un peu moins de 1 % des points de charge sont hors itinerance

In [ ]:
data = irve.loc[:, champs]
data[['id_station_itinerance','id_pdc_itinerance']] = data[['id_station_itinerance','id_pdc_itinerance']].astype('string')
#data[['last_modified','date_maj']] = data[['last_modified','date_maj']].astype('datetime64')
data['non_concerne'] = data['id_station_itinerance'].str.contains('oncern') | data['id_pdc_itinerance'].str.contains('oncern')

non_concerne = data[data['non_concerne']].loc[:, champs].reset_index()['index']
itinerance = data[~data['non_concerne']].loc[:, champs].reset_index()
itinerance = itinerance.loc[:, relations]
print('nombre de pdc hors itinerance : ', len(non_concerne))
print('nombre de pdc en itinerance   : ', len(itinerance))

In [ ]:
il = Ilist.obj(itinerance)
print(il.tree())

-----------------------
## 4 - Bilan initial intégrité
- 36 % des lignes sont erronées ( 18 116 )

In [ ]:
res = analyse_integrite(itinerance)
itinerance_ok_1 = itinerance.loc[itinerance.ok, relations].reset_index(drop=True)
itinerance_1 = itinerance.loc[~itinerance.ok, relations].reset_index(drop=True)
itinerance = itinerance.loc[:, relations]
print("\nnombre d'enregistrements sans erreurs : ", len(itinerance_ok_1))
print("nombre d'enregistrements avec au moins une erreur : ", len(itinerance_1))
print("taux d'erreur : ", round(len(itinerance_1) / len(itinerance) * 100), ' %')

-----------------------
## 5 - Séparation doublons pdc - date de maj
- la moitié des pdc en erreur sont liées aux doublons de pdc
- la suppression des doublons permet de diviser par 5 le nombre de lignes erronnées ( 2 166 )

In [ ]:
itinerance_1['doublons_pdc'] = itinerance_1.sort_values(by='date_maj').duplicated('id_pdc_itinerance', keep='last')

doublons_pdc = itinerance_1[itinerance_1['doublons_pdc']].loc[:, relations].reset_index(drop=True)['index']
itinerance_2 = itinerance_1[~itinerance_1['doublons_pdc']].loc[:, relations].reset_index(drop=True)
itinerance_1 = itinerance_1.loc[:, relations]
print('nombre de doublons pdc : ', len(doublons_pdc))
print('nombre de pdc sans doublon   : ', len(itinerance_2))

In [ ]:
res = analyse_integrite(itinerance_2)
itinerance_ok_3 = itinerance_2.loc[itinerance_2.ok, relations].reset_index(drop=True)
itinerance_3 = itinerance_2.loc[~itinerance_2.ok, relations].reset_index(drop=True)
itinerance_2 = itinerance_2.loc[:, relations]
print("\nnombre d'enregistrements sans erreurs : ", len(itinerance_ok_3))
print("nombre d'enregistrements avec au moins une erreur : ", len(itinerance_3))

-----------------------
## 6 - Cohérence date maj stations
- 20% des erreurs résiduelles sont liées au mélange d'anciens et de nouveaux pdc
- la suppression des anciens pdc permet de réduire de 25% le nombre de lignes erronnées ( 1 616 )
- les dernières erreurs correspondent à 468 stations associées à 32 opérateurs et sont liées à des causes multiples

In [ ]:
itinerance_3['stat_maj'] = itinerance_3.id_station_itinerance + itinerance_3.date_maj
stat_maj_unique = itinerance_3.sort_values(by='stat_maj').drop_duplicates('id_station_itinerance', keep='last')
itinerance_3['last_stat_maj'] = itinerance_3['stat_maj'].isin(stat_maj_unique['stat_maj'])

doublons_stat_maj = itinerance_3[~itinerance_3['last_stat_maj']].loc[:, relations].reset_index(drop=True)['index']
itinerance_4 = itinerance_3[itinerance_3['last_stat_maj']].loc[:, relations].reset_index(drop=True)
itinerance_3 = itinerance_3.loc[:, relations]
print('nombre de doublons stations : ', len(doublons_stat_maj))
print('nombre de pdc sans doublon   : ', len(itinerance_4))

In [ ]:
res = analyse_integrite(itinerance_4)
itinerance_ok_5 = itinerance_4.loc[itinerance_4.ok, relations].reset_index(drop=True)
itinerance_5_full = itinerance_4.loc[~itinerance_4.ok].reset_index(drop=True)
itinerance_5 = itinerance_5_full.loc[:, relations]
#itinerance_4 = itinerance_4.loc[:, relations]
print("\nnombre d'enregistrements sans erreurs : ", len(itinerance_ok_5))
print("nombre d'enregistrements avec au moins une erreur : ", len(itinerance_5))

In [ ]:
il = Ilist.obj(itinerance_5)
print(il.tree())

In [ ]:
# nb maxi d'erreurs
itinerance_4['somme'] = 0
for name in res.keys():
    itinerance_4['somme'] += 1 - itinerance_4[name]
erreurs = max(itinerance_4['somme'])
maxi = itinerance_4[itinerance_4.somme >= erreurs]
print("nombre d'enregistrements avec ", erreurs, " erreurs : ", len(maxi))

-----------------------
## 7 - Synthèse


In [ ]:
#itinerance_5_full.to_csv('erreurs_IRVE.csv')

itinerance['doublons_stat_maj'] = itinerance['index'].isin(doublons_stat_maj)
itinerance['doublons_pdc'] = itinerance['index'].isin(doublons_pdc)
itinerance['lignes_a_corriger'] = itinerance['index'].isin(itinerance_5['index'])
itinerance['doublons_a_supprimer'] = itinerance['doublons_stat_maj'] | itinerance['doublons_pdc']
itinerance['lignes_ko'] = itinerance['doublons_a_supprimer'] | itinerance['lignes_a_corriger']
print('total des lignes à corriger : ', itinerance['lignes_a_corriger'].sum())
print('total des doublons à supprimer : ', itinerance['doublons_a_supprimer'].sum())

In [ ]:
itinerance_ok = itinerance[~itinerance['lignes_ko']].loc[:, relations].reset_index(drop=True)
print('nombre de pdc : ', len(itinerance_ok))
print('\nbilan intégrité :')
res = analyse_integrite(itinerance_ok)

-----------------------
## 8 - Exemples d erreurs résiduelles 

### Erreurs multiples 
- exemple avec le nombre maximal d'erreurs (4) (2 stations soit 12 pdc avec : adresse, nom, nbre_pdc et deux-roues erronés)

In [ ]:
maxi.loc[:, relations]

### Cohérence implantation_station - id_station
- 23 pdc sont liés à une erreur de choix d'implantation (3 stations, 1 opérateur)

In [ ]:
itinerance_4.loc[~itinerance_4['implantation_station - id_station_itinerance'], relations]

### Cohérence nom_station - id_station
- 40 pdc sont associés à une station avec un nom non cohérent

In [ ]:
itinerance_4.loc[~itinerance_4['nom_station - id_station_itinerance'], relations]

### Cohérence adresse - coordonnées
- 414 pdc ont une adresse non cohérente avec les coordonnées géographiques

In [ ]:
itinerance_4.loc[~itinerance_4['adresse_station - coordonneesXY'], relations]

- exemple : une station avec plusieurs adresses -> erreur de saisie ?

In [ ]:
itinerance_5.loc[itinerance_5.coordonneesXY	 == '[0.193942, 49.544211]', relations]

- exemple : une station avec plusieurs adresses -> erreur de saisie ?

In [ ]:
itinerance_5.loc[itinerance_5.coordonneesXY	 == '[2.87930851314442, 48.94679007929618]', relations]

### Cohérence station - coordonnées
- 216 pdc sont associés à des stations avec plusieurs adresses

In [ ]:
itinerance_4.loc[~itinerance_4['coordonneesXY - id_station_itinerance'], relations]

- exemple : une station avec plusieurs coordonnées -> incompréhension distinction station / pdc

In [ ]:
itinerance_5.loc[itinerance_5.id_station_itinerance == 'FR073PCAMAIEUFR', relations]

- exemple : une station avec plusieurs coordonnées -> erreur de saisie

In [ ]:
itinerance_5.loc[itinerance_5.id_station_itinerance == 'FRV75PPX0704', relations]

- exemple : une station par point de charge avec nbre_pdc = 1

In [ ]:
itinerance_4.loc[itinerance_4.coordonneesXY == '[0.191899, 49.506711]', relations]

-----------------------
# Annexe : Initialisation du modèle de données 'mermaid'

In [1]:
from base64 import b64encode
from IPython.display import Image, display

def mermaid(graph):
  display(Image(url="https://mermaid.ink/img/" + b64encode(graph.encode("ascii")).decode("ascii")))

In [2]:
# modèle de données suivant l'application 'mermaid' 
irve = """ 
erDiagram
    AMENAGEUR ||..|{ STATION : amenage
    AMENAGEUR {
        string nom_amenageur
        string siren_amenageur
        string contact_amenageur 
    }
    OPERATEUR ||..|{ STATION : "exploite pour le compte de l enseigne"
    OPERATEUR {
        string contact_operateur PK "M"
        string nom_operateur 
        string telephone_operateur 
    }
    ENSEIGNE ||..|{ STATION : "heberge"
    ENSEIGNE {
        string nom_enseigne PK "M" 
    }
    STATION {
        string  id_station_itinerance PK "M"
        string  nom_station "M"
        enum    implantation_station "M"
        integer nbre_pdc "M"
        string  condition_acces "M"
        string  horaires "M"
        boolean station_deux_roues "M"
        string  id_station_local
        enum    raccordement
        string  num_pdl
        date    date_mise_en_service 
    }
    LOCALISATION ||--|{ STATION : "localise"
    LOCALISATION {
       array   coordonneesXY PK "M"
       string  adresse_station "M"
       string  code_insee_commune 
    }
    STATION ||--|{ POINT_DE_CHARGE : regroupe
    POINT_DE_CHARGE {
        string id_pdc_itinerance PK "M Root"
        number puissance_nominale "M"
        boolean prise_type_ef "M"
        boolean prise_type_2 "M"
        boolean prise_type_combo_ccs "M"
        boolean prise_type_chademo "M"
        boolean prise_type_autre "M"
        boolean paiement_acte "M"
        boolean paiement_autre "M"
        boolean reservation "M"
        enum    accessibilite_pmr "M"
        string  restriction_gabarit "M"
        date    date_maj "M"
        string  id_pdc_local
        boolean gratuit
        boolean paiement_cb
        string  tarification
        string  observations
        boolean cable_t2_attache 
    }
"""